<a href="https://colab.research.google.com/github/DUNGTK2004/Computer-Vision-with-Pytorch/blob/main/1_object_counting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to Computer Vision with PyTorch (p1)
## -- *Object Counting with CNNs* --


---
**Toturial Description**


*   Deep Learning Libraries Used:

  *   pytorch, torchvision, pytorch_ligntning, torchmetrics, cv2









### 1. Import

In [1]:

# !pip install torch
# !pip install torchtext
# !pip install torchvision
# !pip install pytorch-lightning
# !pip install pytorch-lightning-bolts
# !pip install torchmetrics
# !pip install matplotlib
# !pip install numpy
# !pip install ipywidgets
!pip install opencv-python-headless opencv-python
!pip install matplotlib==3.6.0 numpy==1.24.1
!pip install torch==1.12.1 torchtext==0.13.1 torchvision==0.13.1
!pip install pytorch-lightning
!pip install pytorch-lightning-bolts
!pip install torchmetrics
!pip install ipywidgets

In [6]:
!pip install opencv-python-headless opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 11.1 MB/s eta 0:00:00


In [7]:
import torch
torch.__version__

'2.4.0+cu121'

In [8]:
# Deep learning imports.
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, TensorDataset

import torchvision
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks, make_grid
from torchvision.ops import masks_to_boxes
import torchvision.transforms.functional as TF
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

import torchmetrics

# Standard imports.
from typing import List, Union
import gc
import matplotlib.pyplot as plt
import numpy as np
import cv2

# Necessary for creating our images.
from skimage.draw import line_aa

# Interactive widgets for data viz
import ipywidgets as widgets
from ipywidgets import interact

### 2. Pytorch Dataset

In [ ]:
class Draw:
  # Class used to draw shapes onto images
  def __init__(self, img_size, rng):

    self.img_size = img_size
    self.rng = rng

    return None
  def rectangle(self):
    '''
    Returns the image coordinates of a rectangle.
    '''
    # Min and Max rectangle height.
    a = self.rng.integers(self.img_size * 3 / 20, self.img_size * 7 / 20)
    b = self.rng.integers(self.img_size * 3 / 20, self.img_size * 7 / 20)

    # Initial coordinates of the rectangle
    xx, yy = np.where(np.ones((a, b)) == 1)

    # Place the rectangle randomly in the image.
    cx = self.rng.integers(0 + a, self.img_size - a)
    cy = self.rng.integers(0 + b, self.img_size - b)

    rectangle = xx + cx, yy + cy

    return rectangle
  def line(self):
    '''
    Returns the image coordinates of a line
    '''
    # Randomly choose the start and end coordinates
    a, b = self.rng.integers(0, self.img_size / 3, size=2)
    c, d = self.rng.integers(self.img_size / 2, self.img_size, size=2)

    # Flip a coin to see if slope of line is + or -
    coin_flip = self.rng.integers(low=0, hign=2)
    # Use a skimage.draw method to draw the line
    if coin_flip:
      xx, yy, _ = line_aa(a, b, c, d)
    else:
      xx, yy, _ = line_aa(a, d, c, b)
    line = xx, yy

    return line
  def donut(self):
    '''
    Return the image coordinates of an elliptical donut
    '''
    # Define a grid
    xx, yy = np.mgrid[: self.img_size, : self.img_size]
    cx = self.rng.integers(0, self.img_size)
    cy = self.rng.integers(0, self.img_size)

    # Define the width of the donut.
    width = self.rng.uniform(self.img_size / 3, self.img_size)

    # Give the donut some elliptical nature
    e0 = self.rng.uniform(0.1, 5)
    e1 = self.rng.uniform(0.1, 5)

    # Use the forumula for an ellipse
    ellipse = e0 * (xx - cx) ** 2 + e1 * (yy - cy) ** 2

    donut = (ellipse < (self.img_size + width)) & (
        ellipse > (self.img_size - width)
    )

    return donut